https://github.com/agoila/lisa-faster-R-CNN/blob/master/pyimagesearch/nn/conv/resnet.py

In [1]:
import tensorflow as tf

# import the necessary packages
# from keras.layers.normalization import batch_normalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import AveragePooling2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.convolutional import ZeroPadding2D
from keras.layers.core import Activation
from keras.layers.core import Dense
from keras.layers import Flatten
from keras.layers import Input
from keras.models import Model
from keras.layers import add
from keras.regularizers import l2
from keras import backend as K

class ResNet:
	@staticmethod
	def residual_module(data, K, stride, chanDim, red=False,
		reg=0.0001, bnEps=2e-5, bnMom=0.9):
		# the shortcut branch of the ResNet module should be
		# initialize as the input (identity) data
		shortcut = data

		# the first block of the ResNet module are the 1x1 CONVs
		bn1 = tf.keras.layers.BatchNormalization(axis=chanDim, epsilon=bnEps,
			momentum=bnMom)(data)
		act1 = Activation("relu")(bn1)
		conv1 = Conv2D(int(K * 0.25), (1, 1), use_bias=False,
			kernel_regularizer=l2(reg))(act1)

		# the second block of the ResNet module are the 3x3 CONVs
		bn2 = tf.keras.layers.BatchNormalization(axis=chanDim, epsilon=bnEps,
			momentum=bnMom)(conv1)
		act2 = Activation("relu")(bn2)
		conv2 = Conv2D(int(K * 0.25), (3, 3), strides=stride,
			padding="same", use_bias=False,
			kernel_regularizer=l2(reg))(act2)

		# the third block of the ResNet module is another set of 1x1
		# CONVs
		bn3 = tf.keras.layers.BatchNormalization(axis=chanDim, epsilon=bnEps,
			momentum=bnMom)(conv2)
		act3 = Activation("relu")(bn3)
		conv3 = Conv2D(K, (1, 1), use_bias=False,
			kernel_regularizer=l2(reg))(act3)

		# if we are to reduce the spatial size, apply a CONV layer to
		# the shortcut
		if red:
			shortcut = Conv2D(K, (1, 1), strides=stride,
				use_bias=False, kernel_regularizer=l2(reg))(act1)

		# add together the shortcut and the final CONV
		x = add([conv3, shortcut])

		# return the addition as the output of the ResNet module
		return x

	@staticmethod
	def build(width, height, depth, classes, stages, filters,
		reg=0.0001, bnEps=2e-5, bnMom=0.9, dataset="cifar"):
		# initialize the input shape to be "channels last" and the
		# channels dimension itself
		inputShape = (height, width, depth)
		chanDim = -1

		# if we are using "channels first", update the input shape
		# and channels dimension
		if K.image_data_format() == "channels_first":
			inputShape = (depth, height, width)
			chanDim = 1

		# set the input and apply BN
		inputs = Input(shape=inputShape)
		x = tf.keras.layers.BatchNormalization(axis=chanDim, epsilon=bnEps,
			momentum=bnMom)(inputs)

		# check if we are utilizing the CIFAR dataset
		if dataset == "cifar":
			# apply a single CONV layer
			x = Conv2D(filters[0], (3, 3), use_bias=False,
				padding="same", kernel_regularizer=l2(reg))(x)

		# check to see if we are using the Tiny ImageNet dataset
		elif dataset == "tiny_imagenet":
			# apply CONV => BN => ACT => POOL to reduce spatial size
			x = Conv2D(filters[0], (5, 5), use_bias=False,
				padding="same", kernel_regularizer=l2(reg))(x)
			x = tf.keras.layers.BatchNormalization(axis=chanDim, epsilon=bnEps,
				momentum=bnMom)(x)
			x = Activation("relu")(x)
			x = ZeroPadding2D((1, 1))(x)
			x = MaxPooling2D((3, 3), strides=(2, 2))(x)

		# loop over the number of stages
		for i in range(0, len(stages)):
			# initialize the stride, then apply a residual module
			# used to reduce the spatial size of the input volume
			stride = (1, 1) if i == 0 else (2, 2)
			x = ResNet.residual_module(x, filters[i + 1], stride,
				chanDim, red=True, bnEps=bnEps, bnMom=bnMom)

			# loop over the number of layers in the stage
			for j in range(0, stages[i] - 1):
				# apply a ResNet module
				x = ResNet.residual_module(x, filters[i + 1],
					(1, 1), chanDim, bnEps=bnEps, bnMom=bnMom)

		# apply BN => ACT => POOL
		x = tf.keras.layers.BatchNormalization(axis=chanDim, epsilon=bnEps,
			momentum=bnMom)(x)
		x = Activation("relu")(x)
		x = AveragePooling2D((8, 8))(x)

		# softmax classifier
		x = Flatten()(x)
		x = Dense(classes, kernel_regularizer=l2(reg))(x)
		x = Activation("softmax")(x)

		# create the model
		model = Model(inputs, x, name="resnet")

		# return the constructed network architecture
		return model

2022-12-07 06:00:02.404480: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-07 06:00:02.529176: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-07 06:00:02.529203: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-12-07 06:00:03.664286: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-

In [3]:
import cv2
from imutils import build_montages
from keras.datasets import mnist
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD
import matplotlib; matplotlib.use("Agg")
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# import tensorflow_datasets as tfds

# from models.pyimagesearch import resnet
# from keras.layers.normalization import batch_normalization

def load_az_dataset(dataset_path):
    data = []
    labels = []

    # loop over the rows of the A-Z handwritten digit dataset
    for row in open(dataset_path):
        # parse the label and image from the row
        row = row.split(",")
        # first column for label
        label = int(row[0])
        # the rest (784 columns) for image
        image = np.array([int(x) for x in row[1:]], dtype="uint8")

        # images are represented as single channel (grayscale) images
        # that are 28x28=784 pixels -- we need to take this flattened
        # 784-d list of numbers and reshape them into a 28x28 matrix
        image = image.reshape((28, 28))

        data.append(image)
        labels.append(label)
    
    data = np.array(data, dtype='float32')
    labels = np.array(labels, dtype='int')

    return (data, labels)


def load_mnist_dataset():
    ((train_data, train_labels), (test_data, test_labels)) = mnist.load_data()
    data = np.vstack([train_data, test_data])
    labels = np.hstack([train_labels, test_labels])

    return (data, labels)

PATH = 'data/A_Z Handwritten Data.csv'

print("[INFO] loading datasets...")
(az_data, az_labels) = load_az_dataset(PATH)
(digits_data, digits_labels) = load_mnist_dataset()

[INFO] loading datasets...
11490434/11490434 [==============================] - 8s 1us/step


In [4]:
az_labels += 10

data = np.vstack([az_data, digits_data])
labels = np.hstack([az_labels, digits_labels])

print(data.shape, labels.shape)

data = [cv2.resize(image, (32, 32)) for image in data]
data = np.array(data, dtype='float32')

data = np.expand_dims(data, axis=-1)
data /= 255.0

(442451, 28, 28) (442451,)


In [12]:
le = LabelBinarizer()
labels = le.fit_transform(labels)
counts = labels.sum(axis=0)

class_totals = labels.sum(axis=0)
class_weight = {}

for i in range(0, len(class_totals)):
    class_weight[i] = class_totals.max() / class_totals[i]

(train_x, test_x, train_y, test_y) = train_test_split(
    data, labels, test_size=0.20, stratify=labels, random_state=42
)

# construct the image generator for data augmentation
aug = ImageDataGenerator(
    rotation_range=10,
    zoom_range=0.05,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.15,
    horizontal_flip=False,
    fill_mode='nearest'
)
print(le.classes_)
EPOCHS = 10
INIT_LR = 1e-1
BS = 128

opt = SGD(lr=INIT_LR, decay=INIT_LR/EPOCHS)
model = ResNet.build(32, 32, 1, len(le.classes_), (3, 3, 3), (64, 64, 128, 256),
                     reg=0.0005)
model.compile(loss="categorical_crossentropy", optimizer=opt,
              metrics=['accuracy'])

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35]


/home/ciheul/Projects/ml/env/lib/python3.10/site-packages/keras/optimizers/optimizer_v2/gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [13]:
print("[INFO] training network...")
H = model.fit(
    aug.flow(train_x, train_y, batch_size=BS),
    validation_data=(test_x, test_y),
    steps_per_epoch=len(train_x) // BS,
    epochs=EPOCHS,
    class_weight=class_weight,
    verbose=1
)

[INFO] training network...
Epoch 1/3
2765/2765 [==============================] - ETA: 0s - loss: 3.5768 - accuracy: 0.8090

2022-12-07 09:35:39.156674: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 362459136 exceeds 10% of free system memory.


2765/2765 [==============================] - 3210s 1s/step - loss: 3.5768 - accuracy: 0.8090 - val_loss: 0.6461 - val_accuracy: 0.8865
Epoch 2/3
2765/2765 [==============================] - 3208s 1s/step - loss: 1.8737 - accuracy: 0.8797 - val_loss: 0.5991 - val_accuracy: 0.8911
Epoch 3/3
2765/2765 [==============================] - 3233s 1s/step - loss: 1.6888 - accuracy: 0.8885 - val_loss: 0.5823 - val_accuracy: 0.8884


In [18]:
label_names = "0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ"
label_names = [l for l in label_names]

print("[INFO] evaluating network...")
predictions = model.predict(test_x, batch_size=BS)
print(classification_report(test_y.argmax(axis=1),
                            predictions.argmax(axis=1),
                            target_names=label_names))

[INFO] evaluating network...
692/692 [==============================] - 146s 210ms/step
              precision    recall  f1-score   support

           0       0.18      0.77      0.30      1381
           1       0.97      0.96      0.97      1575
           2       0.85      0.90      0.88      1398
           3       0.93      0.98      0.95      1428
           4       0.80      0.91      0.85      1365
           5       0.49      0.82      0.61      1263
           6       0.89      0.95      0.92      1375
           7       0.90      0.96      0.93      1459
           8       0.84      0.95      0.89      1365
           9       0.91      0.97      0.94      1392
           A       0.97      0.97      0.97      2774
           B       0.96      0.94      0.95      1734
           C       0.98      0.96      0.97      4682
           D       0.89      0.91      0.90      2027
           E       0.99      0.95      0.97      2288
           F       0.91      0.99      0.95    

In [19]:
# save the model to disk
print("[INFO] serializing network...")
model.save("handwriting", save_format="h5")

# construct a plot that plots and saves the training history
N = np.arange(0, EPOCHS)
plt.style.use("ggplot")
plt.figure()
plt.plot(N, H.history["loss"], label="train_loss")
plt.plot(N, H.history["val_loss"], label="val_loss")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
# plt.savefig(args["plot"])

[INFO] serializing network...


In [23]:
print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)

(88491, 32, 32, 1)
(88491, 36)


In [20]:
images = []

for i in np.random.choice(np.arange(0, len(test_y)), size=(49,)):
    probs = model.predict(test_x[np.newaxis, i])
    prediction = probs.argmax(axis=1)
    label = label_names[prediction[0]]

    image = (test_x * 255).astype("uint8")
    color = (0, 255, 0)

    if prediction[0] != np.argmax(test_y[i]):
        color = (0, 0, 255)
    
    image = cv2.merge([image] * 3)
    image = cv2.resize(image, (96, 96), interpolation=cv2.INTER_LINEAR)
    cv2.putText(image, label, (5, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.75, color, 2)

    images.append(image)

montage = build_montages(images, (96, 96), (7, 7))[0]
cv2.imshow("OCR Results", montage)
cv2.waitKey(0)

1/1 [==============================] - 0s 119ms/step


error: OpenCV(4.6.0) /io/opencv/modules/imgproc/src/resize.cpp:3689: error: (-215:Assertion failed) !dsize.empty() in function 'resize'
